In [31]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.metrics import confusion_matrix, make_scorer, precision_score, recall_score, f1_score
import seaborn as sns
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt



newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

X_train = newsgroups_train.data #liste de toute les données train
X_test = newsgroups_test.data #liste de toute les données text
y_train = newsgroups_train.filenames #liste de tous les noms de texte
y_test = newsgroups_test.filenames #liste de tous les noms de texte
z = (list(newsgroups_train.target_names))
z #show newsgroup categories

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [32]:
print('Train list contain ', len(y_train), 'articles') #Explore : numbers of articles
print('Test list contain ', len(y_test), 'articles') #Explore : numbers of articles


Train list contain  11314 articles
Test list contain  7532 articles


In [33]:
print(X_train[1988]) #Explore : example of an article

From: smithw@col.hp.com (Walter Smith)
Subject: Re: Playoff predictions
Organization: Colorado Springs IT Center
Lines: 41
NNTP-Posting-Host: fajita19.cs.itc.hp.com

OK, I'll join in the fun and give my playoff predictions: 

1st round: 
----------

PITT vs NYI:  PITT in 4.  
WASH vs NJD:  WASH in 6. 

BOS  vs BUF:  BOS  in 5. 
QUE  vs MON:  MON  in 7. 

CHI  vs STL:  CHI in 4. 
DET  vs TOR:  DET in 6. 

VAN  vs WIN:  WIN in 6. 
CAL  vs  LA:  CAL in 5. 

2nd round: 
----------

PITT vs WASH: PITT in 4. 
BOS  vs MON:  BOS  in 6. 

CHI  vs DET:  CHI  in 7. 
WIN  vs CAL:  CAL  in 5. 

3rd round: 
----------

PITT vs BOS:  PITT in 5. 
CHI  vs CAL:  CHI  in 5. 

Finals:
------

PITT vs CHI: PITT in 5. 


Walter




2) Vectorize text to allow clustering by Kmeans which requiere numerical values

First TfidVectorizer

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer


TfidV =  TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english')
TfidV_train = TfidV.fit_transform(X_train)
TfidV_test = TfidV.fit_transform(X_test)
print(TfidV_train.shape)
print(TfidV_test.shape)

(11314, 56121)
(7532, 41432)


Second vectorization method = CountVectorizer

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

CountV =  CountVectorizer()
CountV_train = CountV.fit_transform(X_train)
CountV_test = CountV.fit_transform(X_test)
print(CountV_train.shape)
print(CountV_test.shape)

(11314, 130107)
(7532, 93564)


In [36]:
def clust_metrics(estimator, data, y = None): #function to evaluate clustering model

    from sklearn.metrics import rand_score, adjusted_rand_score
    from sklearn.metrics import ( mutual_info_score, normalized_mutual_info_score, adjusted_mutual_info_score)
    from sklearn.metrics import ( homogeneity_score, completeness_score, v_measure_score)
    from sklearn.metrics import fowlkes_mallows_score
    from sklearn.metrics import silhouette_score
    from sklearn.metrics import calinski_harabasz_score
    from sklearn.metrics import davies_bouldin_score
    import pandas as pd

    estimator.fit(data)

    metrics_GT = [rand_score, adjusted_rand_score, mutual_info_score, normalized_mutual_info_score, 
               adjusted_mutual_info_score, homogeneity_score, completeness_score, v_measure_score, 
               fowlkes_mallows_score]
    
    metrics_unk = [ silhouette_score]

    score = []
    
    if y is not None :
        for n in metrics_GT :
            met = n(y, estimator.labels_)
            score.append(round(met, 2))

        for n in metrics_unk :
            met = n(data, estimator.labels_)
            score.append(round(met, 2))
    
        return pd.DataFrame({"Clustering Metrics": score}, index=['Rand_Index', 'Adjusted_Rand_Index', 'Mutual_Information', 'Normalized_Mutual_Information', 
               'Adjusted_Mutual_Information', 'Homogeneity_score', 'Completeness_score', 'V_measure_score', 
               'Fowlkes_Mallows_score','Silhouette_score'])
    
    else :
        for n in metrics_unk :
            met = n(data, estimator.labels_)
            score.append(round(met, 2))

        return pd.DataFrame({"Clustering Metrics": score}, index=['Silhouette_score'])

3) Training Kmeans clustering model, and evaluate Clustering

In [37]:
from sklearn.cluster import KMeans

model_Tfid = KMeans(20,n_init= 10).fit(TfidV_train)

Kmeans_score_Tfid = clust_metrics(model_Tfid, TfidV_test, y_test)

In [38]:

model_CountV = KMeans(20,n_init= 10).fit(CountV_train)

Kmeans_score_CountV = clust_metrics(model_CountV, CountV_test, y_test)

Création d'un tableau Contenant les scores pour chaque méthode de corrélation.

In [39]:
Kmeans_score = Kmeans_score_Tfid.rename(columns = {'Clustering Metrics':'Tfid Vectorizer'})


pdToList = list(Kmeans_score_CountV['Clustering Metrics'])


Kmeans_score['CounterVect'] = pdToList

Kmeans_score

,Tfid Vectorizer,CounterVect
Rand_Index,0.84,0.61
Adjusted_Rand_Index,0.00,0.00
Mutual_Information,2.38,1.18
Normalized_Mutual_Information,0.42,0.23
Adjusted_Mutual_Information,0.00,0.00
Homogeneity_score,0.27,0.13
Completeness_score,1.00,1.00
V_measure_score,0.42,0.23
Fowlkes_Mallows_score,0.00,0.00
Silhouette_score,0.01,0.13


Evaluation metrics still very poor. But we can observe a slight increase for some score with the Tfid Vectorization method.
Note that 2 metrics 'Calinski_Harabasz_score' and 'Davies_Bouldin_score' return error on calculation and were remove for the evaluation function.

Next I'm gonna to clean the text from repeated information such as headers and footers informations.

3) Test other clustering method, specified to text clustering : "LDA"

In [45]:
from sklearn.decomposition import LatentDirichletAllocation

model_LDA = LatentDirichletAllocation(n_components=20)

model_LDA.fit(TfidV_train)



LatentDirichletAllocation(n_components=20)

In [47]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

X_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))


y_pred = model_LDA.predict(TfidV_test)

y_true = TfidV_test.target
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

report = classification_report(y_true, y_pred, target_names= z)
print(report)

AttributeError: 'LatentDirichletAllocation' object has no attribute 'predict'